In [1]:
!python -V

Python 3.13.2


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error

In [5]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

2025/06/01 01:16:10 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1748740570042, experiment_id='1', last_update_time=1748740570042, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    return df

In [7]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet')

In [8]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
import xgboost as xgb

In [11]:
from pathlib import Path

In [12]:
models_folder = Path('models')
models_folder.mkdir(exist_ok=True)

In [15]:
with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=30,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/root/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [01:29:22] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:8.73788
[1]	validation-rmse:8.22960
[2]	validation-rmse:7.78914
[3]	validation-rmse:7.40823
[4]	validation-rmse:7.08398
[5]	validation-rmse:6.80130
[6]	validation-rmse:6.56559
[7]	validation-rmse:6.35942
[8]	validation-rmse:6.18716
[9]	validation-rmse:6.04364
[10]	validation-rmse:5.91994
[11]	validation-rmse:5.81441
[12]	validation-rmse:5.72701
[13]	validation-rmse:5.65236
[14]	validation-rmse:5.58821
[15]	validation-rmse:5.53629
[16]	validation-rmse:5.49451
[17]	validation-rmse:5.45443
[18]	validation-rmse:5.42147
[19]	validation-rmse:5.39347
[20]	validation-rmse:5.37267
[21]	validation-rmse:5.35128
[22]	validation-rmse:5.33257
[23]	validation-rmse:5.31780
[24]	validation-rmse:5.30449
[25]	validation-rmse:5.29542
[26]	validation-rmse:5.28365
[27]	validation-rmse:5.27505
[28]	validation-rmse:5.26725
[29]	validation-rmse:5.26004


/root/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [01:29:30] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/06/01 01:29:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run unruly-shad-400 at: http://localhost:5000/#/experiments/1/runs/4b3a36936b414430be0fe2ad5465b44b
🧪 View experiment at: http://localhost:5000/#/experiments/1
